# Python: Expectancy

Let's try Expectancy. 

Based on:
- https://www.learningmarkets.com/determining-expectancy-in-your-trading/
- http://www.unicorn.us.com/trading/expectancy.html

In [10]:
import psycopg2
import pandas as pd
import plotly.express as px
from IPython.core.display import HTML
from IPython.display import display, Image

In [11]:
# Connect to the database
conn = psycopg2.connect(host="quantdata.collective2.com",database="postgres",user="c2public",password="c2public")

cursor = conn.cursor()

# Get max. date we have in stats 
sql = "select max(date) from historical_stats_modern where statname in ('cARdefault','deltaequityp180');"
cursor.execute(sql)
maxDate = cursor.fetchone()

# Prepare SQL text
sqlTemplate = """select 
    r.systemid,
    c2systems.systemname, 
    ((AvgWin / AvgLoss) - (cast(numloss as float) / numtrades)) as Expectancy,
    annReturn.statval as AnnReturn,
    deltaequityp180.statval as DeltaEquityp180,
    (cast(numwins as float) / numtrades) as WinRatio,
    (cast(numloss as float) / numtrades) as LossRatio,
    AvgWin, 
    AvgLoss,
    cast(AvgWin / AvgLoss as float) as RewardToRisk,
    numtrades, 
    numwins, 
    numloss 
from c2ex_results r  
join c2systems on c2systems.systemid = r.systemid 
-- Select by AnnReturn. Based on Daniil's work.
join historical_stats_modern annReturn 
        on annReturn.systemid = r.systemid 
        and annReturn.date = '{date}'
        and annReturn.statname = 'cARdefault' 
        and annReturn.statval > 0
-- Select by DeltaEquityp180. Based on Daniil's work.        
join historical_stats_modern deltaequityp180 
        on deltaequityp180.systemid = r.systemid 
        and deltaequityp180.date = '{date}'
        and deltaequityp180.statname = 'deltaequityp180' 
        and deltaequityp180.statval > 0
where
-- Select systems live on maxDate (from equities table)
(c2systems.systemid IN ( SELECT distinct(systemid) FROM returnsdatainintervalscleanedskip090 where DateStart = '{maxDate}'))
and numtrades > 100 -- a reasonable amount for stats
and numwins > 0 -- no losers, please
and avgloss > 0 -- divider
;"""

# Format maxDate for SQL
maxDateStr = str(maxDate[0])

# Create an SQL command from the template:
sql = sqlTemplate.format(date = maxDate[0], maxDate = maxDateStr)

# Run SQL
cursor.execute(sql)
dbData = cursor.fetchall()
cursor.close()
print("Date used:",maxDateStr)

Date used: 2021-12-20


In [12]:
# Convert dbData to pandas dataframe.
df = pd.DataFrame(dbData, 
                  columns=['SystemId','Name','Expectancy','AnnReturn','DeltaEquityp180',
                           'WinRatio','LossRatio','AvgWin','AvgLoss','RewardToRisk','NumTrades','NumWins','NumLoss'])

# Select strategies having Expectancy above some value
df = df[df['Expectancy'] > 0.2] 
# Sort by Expectancy descending
df.sort_values(by=['Expectancy'], ascending=False, inplace=True) 
# Round nubers
df = df.round(decimals=3)
display(df)

,SystemId,Name,Expectancy,AnnReturn,DeltaEquityp180,WinRatio,LossRatio,AvgWin,AvgLoss,RewardToRisk,NumTrades,NumWins,NumLoss
53,131825719,Options US,19.141,0.18570,3.50000,0.986,0.014,172.39,9.00,19.154,146,144,2
3,96866107,Brands With Momentum,5.910,0.36530,17.60000,0.568,0.432,1719.97,271.19,6.342,132,75,57
86,135738325,Momentum Futures,3.985,0.77650,15.30000,0.215,0.785,1774.09,371.87,4.771,303,65,238
16,117734561,TQQQ Aspire,2.951,0.53640,47.90000,0.394,0.606,2977.42,837.04,3.557,132,52,80
36,128627295,Interlink Futures,2.831,0.72770,127.70000,0.285,0.715,2530.94,713.84,3.546,442,126,316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,135046330,Premium Capital,0.321,0.61010,33.50000,0.756,0.244,141.97,251.17,0.565,643,486,157
13,116569489,FX Alpha Signal,0.318,0.14260,35.10000,0.746,0.254,4144.17,7241.21,0.572,169,126,43
38,129020902,FOREX VIX,0.249,0.15480,14.70000,0.679,0.321,169.33,296.90,0.570,361,245,116
0,13202557,extreme-os,0.240,0.28070,14.00000,0.727,0.273,827.87,1613.59,0.513,4247,3087,1160


In [14]:
html = df.to_html()
file = open("expectancy.html","w")
file.write(html)
file.close()

[See expectancy.html](https://collective2.github.io/QuantData/Documentation/expectancy.html)

In [15]:
# Show the first 30 strategies here, in this notebook
for index, row in df[0:30].iterrows():
    c2url= "https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid={systemid}".format(systemid = row['SystemId'])
    img = Image(url=c2url)
    display(img)
    line = "{systemname} (#{systemid}) Expectancy: {expect}".format(systemname = row['Name'], systemid = row['SystemId'],expect=row['Expectancy']) 
    print(line)
    display(HTML("<hr/>"))


Options US (#131825719) Expectancy: 19.141


Brands With Momentum (#96866107) Expectancy: 5.91


Momentum Futures (#135738325) Expectancy: 3.985


TQQQ Aspire (#117734561) Expectancy: 2.951


Interlink Futures (#128627295) Expectancy: 2.831


...

GardCap Discretionary (#126454200) Expectancy: 1.276


4Timing Trend ML (#124696549) Expectancy: 1.245


Stock Booster MNQ (#130482646) Expectancy: 1.228


Crystal Ball 1 (#136069737) Expectancy: 1.176


sharp Sharpe ES (#117216629) Expectancy: 1.13
